# Little research to see how to realign a Matrix


In [4]:
import numpy as np
import k3d

I = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]], dtype=float)

def get_plot_from_matrix(M):
    origins = M[3][:3].tolist()*3
    vectors = M[0][:3].tolist()+M[1][:3].tolist()+M[2][:3].tolist()
    colors = [0xff0000, 0xff0000, 0x00ff00, 0x00ff00, 0x0000ff, 0x0000ff]
    return k3d.vectors(origins, vectors, colors=colors)

def get_plot_from_vector(M, vector, color=0x000000):
    origins = M[3][:3].tolist()
    vectors = vector.tolist()
    colors = [color,color]
    return k3d.vectors(origins, vectors, colors=colors)

def get_M_aligned_to_x(M, x):
    x = x/np.linalg.norm(x)
    z = np.cross(x, M[1][:3])
    y = np.cross(z, x)
    return np.array([x.tolist()+[0],y.tolist()+[0],z.tolist()+[0],M[3]], dtype=float)

def get_M_aligned_to_y(M, y):
    y = y/np.linalg.norm(y)
    x = np.cross(y, M[2][:3])
    z = np.cross(x, y)
    return np.array([x.tolist()+[0],y.tolist()+[0],z.tolist()+[0],M[3]], dtype=float)

def get_M_aligned_to_z(M, z):
    z = z/np.linalg.norm(z)
    y = np.cross(z, M[0][:3])
    x = np.cross(y, z)
    return np.array([x.tolist()+[0],y.tolist()+[0],z.tolist()+[0],M[3]], dtype=float)
    

    
Mroot = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,1,1]], dtype=float)
localV = np.array([3,0,0])
localV2 = np.array([3,0.3,0.6])

Malign = get_M_aligned_to_x(I, localV)
Malign[3][:3] = Mroot[3][:3]

#compute root as beeing child of align
MrootLocalToAlign = np.dot(Mroot, np.linalg.inv(Malign))

#rotate align
Malign = get_M_aligned_to_x(Malign, localV2)

#recompute root
MnewRoot = np.dot(MrootLocalToAlign, Malign)

plot = k3d.plot()
plot += get_plot_from_matrix(Mroot)
plot += get_plot_from_vector(Mroot, localV)
plot += get_plot_from_vector(Mroot, localV2, 0x555555)
plot += get_plot_from_matrix(MnewRoot)
plot.display()


Output()

In [5]:
import numpy as np

class RealignMatrix(object):
    
    @staticmethod
    def get_M_aligned_to_x(M, x):
        x = x/np.linalg.norm(x)
        z = np.cross(x, M[1][:3])
        y = np.cross(z, x)
        return np.array([x.tolist()+[0],y.tolist()+[0],z.tolist()+[0],M[3]], dtype=float)

    @staticmethod
    def get_M_aligned_to_y(M, y):
        y = y/np.linalg.norm(y)
        x = np.cross(y, M[2][:3])
        z = np.cross(x, y)
        return np.array([x.tolist()+[0],y.tolist()+[0],z.tolist()+[0],M[3]], dtype=float)

    @staticmethod
    def get_M_aligned_to_z(M, z):
        z = z/np.linalg.norm(z)
        y = np.cross(z, M[0][:3])
        x = np.cross(y, z)
        return np.array([x.tolist()+[0],y.tolist()+[0],z.tolist()+[0],M[3]], dtype=float)
    
    
    def __init__(self, M, local_vectors):
        I = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]], dtype=float)
        
        #get the main axis of the vectors
        self._aligned_axis = np.argmax(local_vectors*local_vectors, axis=1)
        
        #compute all the local aligned matrix :
        self._aligned_matrices = []
        self._local_parent_matrices = []
        for axis, vector in zip(self._aligned_axis, local_vectors):
            matrix = [self.get_M_aligned_to_x, self.get_M_aligned_to_y, self.get_M_aligned_to_z][axis](I, vector)
            matrix[3][:3] = M[3][:3]
            self._aligned_matrices.append(np.dot(M, np.linalg.inv(matrix)))
            
    def solve_from_local_vectors(self, M, local_vectors):
        alignedMatrices = [
            np.dot(
                matrix,
                [self.get_M_aligned_to_x, self.get_M_aligned_to_y, self.get_M_aligned_to_z][axis](M, vector)
            ) 
            for axis, matrix, vector in zip(self._aligned_axis, self._aligned_matrices, local_vectors)
        ]
        
        result_matrix = alignedMatrices[0]
        
        if len(alignedMatrices)>1:
            result_matrix = np.sum(alignedMatrices, axis=0)
            x = result_matrix[0][:3]
            y = result_matrix[1][:3]
            x = x/np.linalg.norm(x)
            y = y/np.linalg.norm(y)
            z = np.cross(x, y)
            y = np.cross(z, x)
            result_matrix = np.array([x.tolist()+[0],y.tolist()+[0],z.tolist()+[0],[0,0,0,1]], dtype=float)
            
        result_matrix[3][:3] = M[3][:3]
        return result_matrix
    
    def solve_from_world_coordinates(self, M, M_world, world_points):
        world_inv = np.linalg.inv(M_world)
        local_vectors = [
            np.dot([p[0],p[1],p[2],1], world_inv)[:3] for p in world_points
        ]
        return self.solve_from_local_vectors(M, local_vectors)
        

M = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,1,1]], dtype=float)
Vs = np.array([[2,.1,.5], [0,2,.5], [0,-2,1]])
Vs2 = np.array([[3,2.1,.5], [0,3,.5], [0,-2,1]])
Vs3 = np.array([[3,2.1,1.5], [0,3,1.5], [0,-2,2]])

plot = k3d.plot()
plot.display()

plot += get_plot_from_matrix(M)
for v in Vs:
    plot += get_plot_from_vector(M, v)
for v in Vs2:
    plot += get_plot_from_vector(M, v, 0x555555)
    
align = RealignMatrix(M,Vs)
newM = align.solve_from_local_vectors(M, Vs2)
plot += get_plot_from_matrix(newM)

newM = align.solve_from_world_coordinates(M, M, Vs3)
plot += get_plot_from_matrix(newM)


Output()